In [1]:
import os
import sys
import shutil
import glob

import math
import numpy as np

import torch
import torch.utils.data as torch_data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from random import shuffle, sample

In [2]:

class LSTMValDataLoader(torch_data.Dataset):
    def __init__(self, data_list ):
        self.data_list = data_list
        self.image_list, self.label_list, self.class_list = [], [], []
        self.read_lists()

    def read_lists(self):
        switch = True

        for item in self.data_list:
            datum = item
            im1 = datum["t1"]
            im2 = datum["t2"]
            im3 = datum["t3"]
            im4 = datum["t4"]
            #image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2], image.shape[3] ))
            self.image_list.append((im1, im2, im3, im4))
            
    def __getitem__(self, index):
        return self.image_list[index]


    def __len__(self):
        return len(self.image_list)


# In[47]:


class LSTMTrainDataLoader(torch_data.Dataset):
    def __init__(self, master_data_list, count=1000 ):
        self.master_data_list = master_data_list
        self.count = count
        self.data_list = sample(self.master_data_list[0], self.count) + sample(self.master_data_list[1], self.count)
        shuffle(self.data_list)
        self.image_list, self.label_list = [], []
        self.read_lists()

    def read_lists(self):
        switch = True

        for item in self.data_list:
            datum = np.load(item)
            im1 = datum[0]
            im2 = datum[1]
            
            #image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2], image.shape[3] ))
            label = datum[2]
            
            self.image_list.append((im1, im2))
            self.label_list.append(label)
            
            del im1
            del im2
            
    def __getitem__(self, index):
        return tuple([self.image_list[index], self.label_list[index]])


    def __len__(self):
        return len(self.image_list)
    
    def shuffle_dataset(self):
        del self.image_list
        del self.label_list
        
        self.image_list, self.label_list = [], []
        self.data_list = sample(self.master_data_list[0], self.count) + sample(self.master_data_list[1], self.count)
        shuffle(self.data_list)
        
        self.read_lists()

In [3]:
data_list_0 = glob.glob('C://Users//dhruv//Development//git//thesis_dl-fnirs//data//multilabel//all//mindfulness/siamese/train/0/*.npy')
data_list_1 = glob.glob('C://Users//dhruv//Development//git//thesis_dl-fnirs//data//multilabel//all//mindfulness/siamese/train/1/*.npy')

train_data_list_0 = data_list_0[:10000]
train_data_list_1 = data_list_1[:10000]

test_data_list_0 = data_list_0[20000:]
test_data_list_1 = data_list_1[20000:]


val_data_list = np.load('C://Users//dhruv//Development//git//thesis_dl-fnirs//data//multilabel//all//mindfulness//siamese/validation/data_siamese_val.npy')

In [4]:
train_dataloader = LSTMTrainDataLoader(
    {0: train_data_list_0, 1: train_data_list_1}, count=len(train_data_list_0)
)
print("Train dataset loaded.")

test_dataloader = LSTMTrainDataLoader(
    {0: test_data_list_0, 1: test_data_list_1}, count=len(test_data_list_0)
)
print("Test dataset loaded.")

val_dataloader = LSTMValDataLoader(
    val_data_list
)
print("Validation dataset loaded.")

Train dataset loaded.
Test dataset loaded.
Validation dataset loaded.


In [8]:
def train(model, dataset_loader, epoch, device, optimizer, criterion):
    model.train()
    running_loss = 0.0
    total = 0
    for i , (data, target) in enumerate(dataset_loader):

        print('Train: [Batch: {} ({:.0f}%)]'
              .format(
                  i + 1,
                  (i + 1)*100/len(dataset_loader)
              ), end='\r')
        
        labels = target.long()
        input1, input2 = data[0].float(), data[1].float() 
        input1, input2, labels = input1.to(device), input2.to(device), labels.to(device)
        optimizer.zero_grad()
        # out1, out2
        outputs = model(input1, input2)
        
        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        total += labels.size(0)


    return running_loss/total


def test(model, dataset_loader, device, criterion):
    model.eval()
    running_loss = 0.0
    total = 0
    correct = 0
    for i , (data, target) in enumerate(dataset_loader):

        print('Test Match: [Batch: {} ({:.0f}%)]'
              .format(
                  i + 1,
                  (i + 1)*100/len(dataset_loader)
              ), end='\r')
        
        labels = target.long()
        input1, input2 = data[0].float(), data[1].float() 
        input1, input2, labels = input1.to(device), input2.to(device), labels.to(device)
        
        # out1, out2
        outputs = model(input1, input2)
        
        _, pred = torch.max(outputs, 1)
        
        correct += (pred == labels).sum()
        
        """
        print(outputs)
        print(pred)
        print(labels)
        print(correct)
        """
        loss = criterion(outputs, labels)

        running_loss += loss.item()
        total += labels.size(0)

    return running_loss/total, 100*correct/total


def validate(model, dataset_loader, device, criterion):
    model.eval()
    correct = 0
    total = 0
    valid_loss = 0

    with torch.no_grad():
        for i, data in enumerate(dataset_loader):
            print('Test: [Batch: {} ({:.0f}%)]'
                  .format(
                      i + 1,
                      (i + 1)*100/len(dataset_loader)
                  ), end='\r')

            
            input1, input2, input3, input4 = data[0].float(), data[1].float(), data[2].float(), data[3].float()
            input1, input2, input3, input4 = input1.to(device), input2.to(device), input3.to(device), input4.to(device)

            output1 = model(input1, input2)
            output2 = model(input1, input3)
            output3 = model(input1, input4)
            
            greater_mask = output3[:, 0].gt(output1[:, 0]).cpu().numpy() & output3[:, 0].gt(output2[:, 0]).cpu().numpy()
                
            """
            valid_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            """
            total += input1.size(0)
            correct += np.sum(greater_mask)

    accuracy = 100 * correct / total
    return accuracy


class ConvLSTMNet(nn.Module):
    def __init__( self, num_classes = 2):
        super(ConvLSTMNet, self).__init__()
        
        self.conv3d1 = nn.Conv3d(in_channels=2, out_channels=15, kernel_size=(5, 2, 2))
        self.bn1 = nn.BatchNorm3d(15)
        self.pool1 = nn.AvgPool3d((5, 1, 1))
        
        self.conv3d2 = nn.Conv3d(in_channels=15, out_channels=30, kernel_size=(5, 1, 1), stride=(5, 2, 2))
        self.bn2 = nn.BatchNorm3d(30)
        self.pool2 = nn.AvgPool3d((1, 1, 1))
        
        
        self.convLSTM2d1 = ConvLSTM2D((4, 10), 15, 8, 1)
        self.convLSTM2d2 = ConvLSTM2D((4, 10), 8, 64, 1)

        self.fc1 = nn.Linear(2560, 128)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 2)
        self.fc5 = nn.Linear(4, 2)

    def sub_forward(self, x, hidden_states= None):

        b_idx, ts, n_ch, w, h = x.size()
        if hidden_states:
            self.h1, self.c1 = hidden_states
        else:
            self.h1, self.c1 = self.convLSTM2d1.init_hidden(batch_size=b_idx)
            self.h2, self.c2 = self.convLSTM2d2.init_hidden(batch_size=b_idx)
        
        # N, C, D, H, W = 1, 1, 160, 5, 22
        out = x.permute(0, 2, 1, 3, 4)
        
        out = self.conv3d1(out)
        out = self.pool1(out)
        out = self.bn1(out)
        """
        out = self.conv3d2(out)
        out = self.pool2(out)
        out = self.bn2(out)
        """
        #print(out.size())
        # N, D, C, H, W = 1, 1, 160, 5, 22
        out = out.permute(0, 2, 1, 3, 4)
        for t in range(0, out.size(1)):
            
            self.h1, self.c1 = self.convLSTM2d1(
                out[:, t, :, :, :], (self.h1, self.c1)
            )

            self.h2, self.c2 = self.convLSTM2d2(
                self.h1, (self.h2, self.c2)
            )

        out = self.h2.view(self.h2.size(0), -1)


        out = self.fc1(out)
        """
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        out = self.fc5(out)
        """
        
        return out
    
    def forward(self, x1, x2, hidden_states = None):
        out1 = self.sub_forward(x1)
        out2 = self.sub_forward(x2)
        
        
        o_sum = out1+out2
        o_sum = torch.sum(o_sum, 1)
        
        o_prod = out1*out2
        o_prod = torch.sum(o_prod, 1)
        
        o_abs = torch.abs(out1-out2)
        o_abs = torch.sum(o_abs, 1)
        
        o_sq = torch.pow(out1-out2, 2)
        o_sq = torch.sum(o_sq, 1)
        
        out = torch.stack((o_sum, o_prod, o_abs, o_sq))
        out = out.permute(1, 0)
        out = self.fc5(out)
        return out

In [9]:
class ConvLSTM2D(nn.Module):
    def __init__(
            self,
            input_size, input_channel, hidden_channel,
            kernel_size, stride=1, padding=0
    ):
        """
        Initializations
        :param input_size: (int, int): height, width tuple of the input
        :param input_channel: int: number of channels of the input
        :param hidden_channel: int: number of channels of the hidden state
        :param kernel_size: int: size of the filter
        :param stride: int: stride
        :param padding: int: width of the 0 padding
        """

        super(ConvLSTM2D, self).__init__()
        self.n_h, self.n_w = input_size
        self.n_c = input_channel
        self.hidden_channel = hidden_channel

        self.conv_xi = nn.Conv2d(
            self.n_c,
            self.hidden_channel,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        self.conv_xf = nn.Conv2d(
            self.n_c,
            self.hidden_channel,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        self.conv_xo = nn.Conv2d(
            self.n_c,
            self.hidden_channel,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        self.conv_xg = nn.Conv2d(
            self.n_c,
            self.hidden_channel,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        self.conv_hi = nn.Conv2d(
            self.hidden_channel,
            self.hidden_channel,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        self.conv_hf = nn.Conv2d(
            self.hidden_channel,
            self.hidden_channel,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        self.conv_ho = nn.Conv2d(
            self.hidden_channel,
            self.hidden_channel,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        self.conv_hg = nn.Conv2d(
            self.hidden_channel,
            self.hidden_channel,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        self.conv_hi = nn.Conv2d(
            self.hidden_channel,
            self.hidden_channel,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

    def forward(self, x, hidden_states):
        """
        Forward prop.
        reference: https://arxiv.org/pdf/1506.04214.pdf (3.1)
        :param x: input tensor of shape (n_batch, n_c, n_h, n_w)
        :param hidden_states: (tensor, tensor) for hidden and cell states.
                              Each of shape (n_batch, n_hc, n_hh, n_hw)
        :return: (hidden_state, cell_state)
        """

        hidden_state, cell_state = hidden_states

        xi = self.conv_xi(x)
        hi = self.conv_hi(hidden_state)
        xf = self.conv_xf(x)
        hf = self.conv_hf(hidden_state)
        xo = self.conv_xo(x)
        ho = self.conv_ho(hidden_state)
        xg = self.conv_xg(x)
        hg = self.conv_hg(hidden_state)

        i = torch.sigmoid(xi + hi)
        f = torch.sigmoid(xf + hf)
        o = torch.sigmoid(xo + ho)
        g = torch.tanh(xg + hg)
        #print(f.size(), cell_state.size(), i.size(), g.size())
        cell_state = f * cell_state + i * g

        hidden_state = o * torch.tanh(cell_state)

        return hidden_state, cell_state

    def init_hidden(self, batch_size):
        return (
                torch.zeros(batch_size, self.hidden_channel, self.n_h, self.n_w).cuda(),
                torch.zeros(batch_size, self.hidden_channel, self.n_h, self.n_w).cuda()
               )


In [11]:

if __name__ == '__main__':

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ConvLSTMNet()

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=10, verbose=True)
    epochs = 30
    curr_epoch = 0

    model.to(device)
    criterion.to(device)

    train_loss_history, test_loss_history = [], []
    test_acc_history, val_acc_history = [], []
    
    
    best_accuracy = 0.0
    best_model = -1

    MODEL_PATH_PREFIX = 'model-convlstm-epoch'
    MODEL_PATH_EXT = 'pth'

    train_loader = torch_data.DataLoader(
        train_dataloader,
        batch_size=128, shuffle=True, num_workers=0
    )
    
    test_loader = torch_data.DataLoader(
        test_dataloader,
        batch_size=256, shuffle=False, num_workers=0
    )

    val_loader = torch_data.DataLoader(
        val_dataloader,
        batch_size=8, shuffle=False, num_workers=0
    )

    
    is_best = True
    best_score = 0
    best_epoch = 0
    
    
    print("Epoch\tTrain Loss\tTest Loss\tTest Accuracy\tValidation Accuracy")
    while curr_epoch <= epochs:
        
        train_running_loss = train(
            model, train_loader,
            curr_epoch, device, optimizer, criterion
        )
        
        test_running_loss, test_accuracy = test(
            model, test_loader,
            device, criterion
        )
        
        val_accuracy = validate(
            model, val_loader,
            device, criterion
        )

        scheduler.step(val_accuracy)
        # record all the models that we have had so far.
        train_loss_history.append(
            train_running_loss
        )

        test_acc_history.append(
            test_accuracy
        )
        
        test_loss_history.append(
            test_running_loss
        )
        
        val_acc_history.append(
            val_accuracy
        )
        # write model to disk.

        state = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        
            
        'models/model-lenet-epoch'
        torch.save(
            state,
            MODEL_PATH_PREFIX + '-{}.'.format(curr_epoch) + MODEL_PATH_EXT
        )

        print('{}\t{:.5f}\t\t{:.5f}\t\t{:.5f}\t\t{:.5f}\t\t'.format(
            curr_epoch,
            train_running_loss,
            test_running_loss,
            test_accuracy,
            val_accuracy
        ))
        curr_epoch+=1

Epoch	Train Loss	Test Loss	Test Accuracy	Validation Accuracy
0	0.00632		0.00277		52.00000		31.94444		
1	0.00544		0.00277		53.00000		36.80556		
2	0.00540		0.00275		54.00000		38.19444		
3	0.00532		0.00272		57.00000		33.33333		


RuntimeError: cublas runtime error : the GPU program failed to execute at C:/a/w/1/s/tmp_conda_3.5_090548/conda/conda-bld/pytorch_1550394496986/work/aten/src/THC/THCBlas.cu:258

In [ ]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)
    
dump_tensors()

In [ ]:
"""
model_path = 'model-convlstm-epoch-18.pth'
model = ConvLSTMNet()
model.load_state_dict(torch.load(model_path)["model"])
model.eval()
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)
"""

In [ ]:
"""
true = []
pred = []
for data, lab in test_dataloader:
    x1 = data[0]
    x2 = data[1]
    x1 = torch.from_numpy(np.array([x1])).float()
    x2 = torch.from_numpy(np.array([x2])).float()
    out = model(x1, x2)

    #0 = matching, 1 = different
    match = torch.max(out.data,  1)[1].numpy()[0]
    true.append(lab)
    pred.append(match)
    #print(lab, "match" if match == 0 else "different")
"""

In [ ]:
# sklearn
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
cnf = confusion_matrix(y_pred=pred, y_true=true)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=[0, 1], yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(true, pred,  classes=["0", "1"], normalize=True,
                      title='Confusion matrix, without normalization')

In [ ]:
0 && 0